In [1]:
from dfs.df_reader import get_asserted_spells_df

spells_df = get_asserted_spells_df(verbose=True)

Spells: 100%|██████████| 202/202 [00:00<00:00, 206.31it/s]


Validating schema.
Schema errors.
    index                           spell_name              failure_case
0       1                      Ataque Certeiro                  1 rodada
1       5                     Controlar Chamas     instantânea ou 1 hora
2       6                         Criar Chamas                10 minutos
3      14                   Flecha Improvisada                10 minutos
4      17                         Ilusão Menor                  1 minuto
5      22                    Lâmina Estrondosa                  1 rodada
6      24                             Mensagem                  1 rodada
7      25                         Moldar Terra     instantâneo ou 1 hora
8      26                          Moldar Água     instantânea ou 1 hora
9      31                      Pedra Encantada                  1 minuto
10     32                      Prestidigitação                até 1 hora
11     33              Proteção contra Lâminas                  1 rodada
12     42        

In [2]:
spells_df

,nome,name,nivel,escola,ritual,elementos,tempo_conjuracao,alcance_area,componentes,mana,duracao,attack_save,dmg_effect,dmg,classes,tags,descricao,source,mana_adicional,magia_rara
0,Amizade,Friends,0,[psíquica],False,[],1 ação,pessoal,SM (uma pequena quantidade de maquiagem),100,"concentração, até 1 minuto",N/A,N/A,N/A,"[arqueiro, bardo, mago, ladino, guerreiro, mon...",[buff],"Pela duração, você terá vantagem em todos os t...",LDJ,10 por turno,False
1,Ataque Certeiro,True Strike,0,[psíquica],False,[],1 ação,9 metros,S,100,1 rodada,N/A,N/A,N/A,"[arqueiro, guerreiro, ladino, mago, monge, xamã]",[buff],"Você encara um alvo no alcance, vasculhando se...",LDJ,N/A,False
2,Ataque Elemental,Custom,0,"[elemental, musical, ilusionista, necromancia,...",False,"[água, terra, fogo, ar, luz, sombras, relâmpag...",1 ação,18 metros,S,0,instantânea,N/A,elemental,1d6,"[arqueiro, bardo, mago, guerreiro, ladino, mon...",[dano],Essa é a magia mais básica de dano que qualque...,Custom,N/A,False
3,Chama Sagrada,Secred Flame,0,[elemental],False,[luz],1 ação,18 metros,VS,150,instantânea,DEX Save,luz,1d8,"[arqueiro, bardo, mago, monge, xamã]",[dano],Radiação similar a uma chama desce sobre uma c...,LDJ,N/A,False
4,Chicote,Lightning Lure,0,[elemental],False,"[relâmpago, luz, sombras, metal, terra, fogo]",1 ação,"4,5 metros",VS,150,instantânea,STR Save,elemental,1d8,"[mago, monge, guerreiro]","[dano, controle]",Você cria um chicote elemental que golpeia uma...,Tasha,N/A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Vento Protetor,Warding Wind,2,[elemental],False,[ar],1 ação,pessoal,V,500,"concentração, até 10 minutos",N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[debuff, defesa]",Um vento forte (30 quilômetros por hora) sopra...,LDJ,5 por minuto,False
198,Ver o Invisível,See Invisibility,2,"[espiritual, necromancia, pura, elemental]",False,[sombras],1 ação,pessoal,VSM (um pouco de talco e um pó de prata polvil...,700,1 hora,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[buff, detecção]","Pela duração, você vê criaturas e objetos invi...",LDJ,N/A,False
199,Visão no Escuro,Darkvision,2,"[elemental, necromancia, espiritual]",False,"[luz, sombras]",1 ação,toque,VSM (ou um pedaço de cenoura seca ou uma ágata),700,8 horas,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[buff, exploração, toque]",Você toca uma criatura voluntária para concede...,LDJ,N/A,False
200,Vínculo Protetor,Warding Bond,2,"[elemental, musical, espiritual]",False,"[água, terra, ar, luz, metal]",1 ação,toque,"VSM (um par de anéis de platina valendo, no mí...",750,1 hora,N/A,N/A,N/A,"[guerreiro, bardo, monge, xamã]","[buff, defesa, toque]",Essa magia protege uma criatura voluntária que...,LDJ,N/A,False


In [3]:
import pandera

In [2]:
spells_df.duracao.unique()

array(['concentração, até 1 minuto', '1 rodada', 'instantânea',
       'instantânea ou 1 hora', '10 minutos', '1 minuto',
       'concentração, até 1 hora', 'instantâneo ou 1 hora', 'até 1 hora',
       'até 1 minuto', '8 horas', '24 horas', '1 hora',
       'concentração, até 10 minutos', 'concentração, até 1 dia',
       '10 dias', 'concentração, até 10 minuto', '2 rodadas',
       'até ser dissipada', 'concentração, 10 minutos', '1 turno',
       'concentração, até 8 horas'], dtype=object)

In [5]:
inferred_schema = pandera.infer_schema(spells_df)

with open("inferred_schema.py", "w") as file:
    file.write(inferred_schema.to_script())

In [6]:
from dfs.DFFilter import filter_df
from spell.SpellPrinter import print_spell, print_spells_by_name, print_spells_for_df

my_filter = {
    "classes": ['guerreiro'],
    'escola': ['musical', 'elemental'],
    'elementos': ['fogo'],
    "nivel": [0, 1],
}
filtered_df = filter_df(spells_df, my_filter)

print_spells_for_df(filtered_df)

ModuleNotFoundError: No module named 'dfs.DFReader'

In [ ]:
filtered_df

,nome,name,nivel,escola,ritual,elementos,tempo_conjuracao,alcance_area,componentes,mana,duracao,attack_save,dmg_effect,dmg,classes,tags,descricao,source,mana_adicional,magia_rara
18,Infestação,Infestation,0,"[invocação, necromancia]",False,[],1 ação,9 metros,VSM (uma pulga viva),100,instantânea,CON Save,veneno,1d6,"[arqueiro, mago, bardo, ladino, xamã]","[dano, controle]","Você provoca uma nuvem de ácaros, pulgas e out...",Xanathar,N/A,False
28,Mãos Mágicas,Mage Hand,0,"[necromancia, invocação]",False,[],1 ação,9 metros,VS,150,instantânea,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, monge, bardo, ma...",[utilidade],"Uma mão espectral flutuante aparece num ponto,...",LDJ,N/A,False
62,Convocar Familiar,Find Familiar,1,[invocação],False,[],1 hora,3 metros,"VSM (carvão, incenso e ervas no valor de 10 po...",700,instantânea,N/A,N/A,N/A,"[arqueiro, mago, xamã]","[dano, utilidade, exploração]","Você adquire os serviços de uma familiar, um e...",LDJ,N/A,False
114,Servo Invisível,Unseen Servant,1,[invocação],True,[],1 ação,18 metros,VSM (um pedaço de barbante e um pouco de madeira),200,1 hora,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[exploração, utilidade]","Essa magia cria uma força invisível, sem mente...",LDJ,N/A,False
138,Convocar Montaria,Find Steed,2,"[invocação, necromancia, espiritual]",False,[],10 minutos,9 metros,VS,1400,instantânea,N/A,N/A,N/A,"[arqueiro, guerreiro, bardo, mago]","[dano, comunicação, exploração]",Você convoca um espírito que assume a forma de...,LDJ,N/A,True
151,Espírito Curativo,Healing Spirits,2,"[elemental, invocação, espiritual]",False,"[água, terra, luz]",1 ação bônus,18 metros,VS,700,"concentração, até 1 minuto",N/A,N/A,1d6,"[bardo, monge, xamã]",[cura],Você invoca um espírito de natureza para confo...,LDJ,15 por turno,False
160,Invocar Fera,Summon Beast,2,"[elemental, invocação]",False,"[água, terra, fogo, ar]",1 ação,18 metros,V,800,1 hora,N/A,N/A,N/A,"[bardo, mago, feiticeiro, bruxo, xamã]",[dano],Você convoca um espírito feral. Ele se manifes...,Tasha,N/A,False
168,Mensageiro Animal,Animal Messenger,2,"[elemental, invocação, musical]",True,[terra],1 ação,9 metros,VSM (um punhado de comida),650,24 horas,N/A,N/A,N/A,"[arqueiro, guerreiro, bardo, monge, mago, xamã]","[utilidade, comunicação]","Através dessa magia, você usa um animal para e...",LDJ,N/A,False
184,Rebanho de Familiares,Flock of Familiars,2,[invocação],False,[],1 minuto,toque,VS,1400,"concentração, até 1 hora",N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[comunicação, detecção, exploração]",Você temporariamente invoca três familiares - ...,Etc.,50 a cada 5 minutos,False
188,Sentido Bestial,Beast Sense,2,"[elemental, musical, invocação]",False,[terra],1 ação,toque,S,500,"concentração, até 1 hora",N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[exploração, toque]",Você toca uma besta voluntária. Pela duração d...,LDJ,20 a cada 5 minutos,False


In [ ]:
from IPython.display import display


def get_mana_stats(spells_df):
    display(spells_df.mana.value_counts().sort_index(ascending=False))
    print(f"Mean =\t{filtered_df.mana.mean():.3f}")
    print(f"Std  =\t{filtered_df.mana.std():.3f}")


get_mana_stats(filtered_df)


1400    2
800     1
700     2
650     1
500     1
200     1
150     1
100     1
Name: mana, dtype: int64

Mean =	660.000
Std  =	462.961


In [ ]:
import dice.DiceCalculator as dice

dice.get_dice_formula_mean('4d8+2')

20.0